In [1]:
import wfdb
import struct
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.signal import welch, butter, filtfilt, savgol_filter, find_peaks

## Functions

### Plotting Functions

In [2]:
def plot_interactive_ecg(
    df: pd.DataFrame,
    leads: list[str],
    start: int = 0,
    end: int = 5000,
    label: str = "ECG Signal",
    width: int = 1200,
    height: int = 500
) -> None:
    """
    Plots an interactive plot using plotly
    """
    # Create a Plotly figure
    fig = go.Figure()

    # Add traces for each lead
    for lead in leads:
        if lead in df.columns:
            fig.add_trace(go.Scatter(
                x=df['time'][start:end],
                y=df[lead][start:end],
                mode='lines',
                name=lead
            ))

    # Update layout
    fig.update_layout(
        title=label,
        xaxis_title='Time (ms)',
        yaxis_title='ECG Signal (mV)',
        legend_title='Leads',
        width=width,  # Adjust width as needed
        height=height,  # Adjust height as needed
    )

    # Show the plot
    fig.show()


# Function to plot ECG data
def plot_ecg_data(
    df: pd.DataFrame,
    label: str = 'ECG Signals Over Time',
    start: int = 0,
    end: int = 1000
) -> None:
    plt.figure(figsize=(20, 7))
    for column in df.columns:
        if column != 'time':
            plt.plot(df['time'][start:end], df[column]
                     [start:end], label=column)

    plt.xlabel('Time (ms)')
    plt.ylabel('ECG Signal (mV)')
    plt.title(label)
    plt.legend()
    plt.grid(True)
    plt.show()


# Function to plot selected ECG leads
def plot_selected_ecg_leads(
    df: pd.DataFrame,
    leads: list[str],
    label: str = 'Selected ECG Signals Over Time',
    start: int = 0,
    end: int = 1000
) -> None:
    plt.figure(figsize=(20, 7))
    for lead in leads:
        if lead in df.columns:
            plt.plot(df['time'][start:end], df[lead][start:end], label=lead)

    plt.xlabel('Time (ms)')
    plt.ylabel('ECG Signal (mV)')
    plt.title(label)
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_selected_ecg_leads_separately(
    df: pd.DataFrame,
    leads: list[str],
    start: int = 0,
    end: int = 1000
) -> None:
    num_leads = len(leads)
    colormap = plt.colormaps['tab10']
    colors = [colormap(i) for i in range(10)]

    fig, axes = plt.subplots(num_leads, 1, figsize=(
        15, 2 * num_leads), sharex=True)

    if num_leads == 1:
        axes = [axes]

    for i, lead in enumerate(leads):
        if lead in df.columns:
            axes[i].plot(
                df['time'][start:end],
                df[lead][start:end],
                label=lead,
                color=colors[i % 10]
            )
            axes[i].set_ylabel('Signal (mV)')
            axes[i].set_title(f'Lead {lead}')
            axes[i].grid(True)

    axes[-1].set_xlabel('Time (ms)')
    plt.tight_layout()
    plt.show()

### Filter Functions

ECG signals often contain various types of noise that can obscure important features. Effective filtering is crucial for accurate analysis and interpretation. Here’s a guide to common types of noise in ECG signals and the appropriate filters to address them.

- Baseline Wander: Low-frequency noise caused by patient movement or respiration.
  - Solution: **High-pass filter** with a cutoff frequency around 0.5-0.7 Hz.

- Powerline Interference: Noise from electrical sources, typically at 50 or 60 Hz.
  - Solution: **Notch filter** at 50/60 Hz or band-stop filter around the powerline frequency.

- Muscle Artifacts: High-frequency noise caused by muscle contractions.
  - Solution: **Low-pass filter** with a cutoff frequency around 40-45 Hz.

- After applying the above three filters, further noise reduction can be achieved by applying the **Savitzky-Golay filter** to smooth out high-frequency noise while preserving signal features.

In [ ]:
# High-Pass Filter to Remove Baseline Wander
def highpass_filter(signal, cutoff, fs, order=1):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal


# Low-Pass Filter to Remove High-Frequency Noise
def lowpass_filter(signal, cutoff, fs, order=1):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal


# Notch Filter to Remove Powerline Interference
def notch_filter(signal, fs, freq=50.0, quality_factor=30.0):
    nyquist = 0.5 * fs
    w0 = freq / nyquist
    b, a = butter(
        2,
        [w0 - w0/quality_factor, w0 + w0/quality_factor],
        btype='bandstop'
    )
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal


# Smooth the signal and preserve morphological features.
def apply_savgol_filter(signal, window_length, polyorder):
    return savgol_filter(signal, window_length, polyorder)


# Filter function to apply all the filters sequentially
def filter_signal(signal, fs):
    # High-pass filter to remove baseline wander
    signal = highpass_filter(signal, cutoff=0.5, fs=fs, order=1)

    # Low-pass filter to remove high-frequency noise
    signal = lowpass_filter(signal, cutoff=45.0, fs=fs, order=1)

    # Notch filter to remove powerline interference (assuming 50 Hz)
    signal = notch_filter(signal, fs=fs, freq=50.0, quality_factor=30.0)

    # Savgol filter to smooth the signal
    signal = savgol_filter(signal, window_length=15, polyorder=3)

    return signal

### Feature Extraction Functions

In [4]:
def detect_r_peaks(ecg_signal, fs):
    # Differentiate
    diff = np.diff(ecg_signal)

    # Squaring
    squared = diff ** 2

    # Moving window integration
    window_size = int(0.046 * fs)  # 0.046 is selected after a lot of testing
    integrated = np.convolve(
        squared,
        np.ones(window_size) / window_size, mode='same'
    )

    # Find R-peaks
    threshold = 0.6 * np.max(integrated)
    r_peaks, _ = find_peaks(
        integrated,
        height=threshold, distance=fs * 0.1
    )

    return r_peaks


def compute_rr_intervals(r_peaks, sample_rate):
    rr_intervals = np.diff(r_peaks) / sample_rate * 1000  # Convert to ms
    return rr_intervals


def get_heart_rate(rr_intervals):
    rr_intervals_sec = rr_intervals / 1000
    heart_rate = 60 / rr_intervals_sec  # In beats per minute (bpm)
    return heart_rate

In [5]:
def compute_hrv_features(rr_intervals):
    rmssd = np.sqrt(np.nanmean(np.square(np.diff(rr_intervals))))
    nn50 = np.sum(np.abs(np.diff(rr_intervals)) > 50)
    pnn50 = nn50 / len(rr_intervals) * 100
    return rmssd, nn50, pnn50

In [6]:
def extract_morphological_features(ecg_signal, r_peaks, sampling_rate):
    p_peaks_indices = []
    q_peaks_indices = []
    s_peaks_indices = []
    t_peaks_indices = []

    p_peaks_amplitudes = []
    q_peaks_amplitudes = []
    r_peaks_amplitudes = []
    s_peaks_amplitudes = []
    t_peaks_amplitudes = []

    qrs_complexes = []
    t_durations = []
    qt_intervals = []
    st_windows = []
    st_segments_deviations = []

    for r_peak in r_peaks:
        qsp_search_window = int(0.16 * sampling_rate)
        t_search_window = int(0.3 * sampling_rate)

        q_peak = np.argmin(
            ecg_signal[max(0, r_peak - qsp_search_window // 2):r_peak]
        ) + max(0, r_peak - qsp_search_window // 2)
        q_peaks_amplitudes.append(ecg_signal[q_peak])

        s_peak = np.argmin(
            ecg_signal[r_peak:min(
                len(ecg_signal), r_peak + qsp_search_window // 2)]
        ) + r_peak
        s_peaks_amplitudes.append(ecg_signal[s_peak])

        p_peak = None
        if len(ecg_signal[max(0, q_peak - qsp_search_window):q_peak]) > 0:
            p_peak = np.argmax(
                ecg_signal[max(0, q_peak - qsp_search_window):q_peak]
            ) + max(0, q_peak - qsp_search_window)
            p_peaks_amplitudes.append(ecg_signal[p_peak])
        

        t_peak = np.argmax(
            ecg_signal[s_peak:min(len(ecg_signal), s_peak + t_search_window)]
        ) + s_peak
        t_peaks_amplitudes.append(ecg_signal[t_peak])

        # Compute ST Segment Deviation
        st_segment_start = s_peak + int(0.02 * sampling_rate)
        # End of ST segment window, but not beyond T peak
        st_segment_end = min(
            t_peak,
            st_segment_start + int(0.08 * sampling_rate)
        )

        st_windows.append((st_segment_start, st_segment_end))

        if st_segment_start < len(ecg_signal) and st_segment_end < len(ecg_signal):
            st_segment_mean = np.nanmean(
                ecg_signal[st_segment_start:st_segment_end])
            st_segment_deviation = st_segment_mean - ecg_signal[r_peak]
            st_segments_deviations.append(st_segment_deviation)
        else:
            # Handle out-of-bound cases with NaN
            st_segments_deviations.append(np.nan)

        p_peaks_indices.append(p_peak)
        q_peaks_indices.append(q_peak)
        s_peaks_indices.append(s_peak)
        t_peaks_indices.append(t_peak)

        r_peaks_amplitudes.append(ecg_signal[r_peak])

        qrs_complexes.append(s_peak - q_peak)
        t_durations.append(t_peak - r_peak)
        qt_intervals.append(t_peak - q_peak)

    return (r_peaks_amplitudes, p_peaks_indices, p_peaks_amplitudes, q_peaks_indices, q_peaks_amplitudes, s_peaks_indices, s_peaks_amplitudes, t_peaks_indices, t_peaks_amplitudes, qrs_complexes, t_durations, qt_intervals, st_segments_deviations, st_windows)

In [7]:
def compute_frequency_features(ecg_signal, sampling_rate):
    frequencies, power_spectral_density = welch(
        ecg_signal, fs=sampling_rate, nperseg=1024
    )
    vlf_band = (0.003, 0.04)
    lf_band = (0.04, 0.15)
    hf_band = (0.15, 0.4)

    vlf_power = np.trapz(
        power_spectral_density[
            (frequencies >= vlf_band[0]) & (frequencies < vlf_band[1])
        ]
    )
    lf_power = np.trapz(
        power_spectral_density[
            (frequencies >= lf_band[0]) & (frequencies < lf_band[1])
        ]
    )
    hf_power = np.trapz(
        power_spectral_density[
            (frequencies >= hf_band[0]) & (frequencies < hf_band[1])
        ]
    )

    return vlf_power, lf_power, hf_power

In [8]:
def extract_features(ecg_signal, fs):
    r_peaks_indices = detect_r_peaks(ecg_signal, fs)
    rr_intervals = compute_rr_intervals(r_peaks_indices, fs)
    heart_rate = get_heart_rate(rr_intervals)
    rmssd, nn50, pnn50 = compute_hrv_features(rr_intervals)

    r_peaks_amplitudes, p_peaks_indices, p_peaks_amplitudes, q_peaks_indices, q_peaks_amplitudes, s_peaks_indices, s_peaks_amplitudes, t_peaks_indices, t_peaks_amplitudes, qrs_complexes, t_durations, qt_intervals, st_segments_deviations, st_windows = extract_morphological_features(
        ecg_signal, r_peaks_indices, fs
    )
    vlf_power, lf_power, hf_power = compute_frequency_features(
        ecg_signal, fs
    )

    st_slopes = []
    for st_segment_start, st_segment_end in st_windows:
        if st_segment_start < len(ecg_signal) and st_segment_end < len(ecg_signal):
            slope = (ecg_signal[st_segment_end] - ecg_signal[st_segment_start]
                     ) / (st_segment_end - st_segment_start)
            st_slopes.append(slope)
        else:
            st_slopes.append(np.nan)  # Handle out-of-bound cases with NaN

    # Replace None with np.nan in durations
    p_durations = np.append(
        [np.nan if x is None else x for x in p_peaks_indices], 
        [np.nan] * (len(r_peaks_indices) - len(p_peaks_indices))
    )
    qrs_durations = np.append(
        [np.nan if x is None else x for x in qrs_complexes],
        [np.nan] * (len(r_peaks_indices) - len(qrs_complexes))
    )
    t_durations = np.append(
        [np.nan if x is None else x for x in t_durations],
        [np.nan] * (len(r_peaks_indices) - len(t_durations))
    )
    qt_intervals = np.append(
        [np.nan if x is None else x for x in qt_intervals], 
        [np.nan] * (len(r_peaks_indices) - len(qt_intervals))
    )

    # print(f"{p_peaks_indices=}")
    # print(f"{p_peaks_amplitudes=}")

    # print(f"{q_peaks_indices=}")
    # print(f"{q_peaks_amplitudes=}")

    # print(f"{r_peaks_indices=}")
    # print(f"{r_peaks_amplitudes=}")

    # print(f"{s_peaks_indices=}")
    # print(f"{s_peaks_amplitudes=}")

    # print(f"{t_peaks_indices=}")
    # print(f"{t_peaks_amplitudes=}")

    dummy = {
        # 'rr_intervals': rr_intervals,
        # 'heart_rate': heart_rate,
        # 'st_segments_deviations': np.array(st_segments_deviations),
        # 'st_segments_deviations': np.array(st_segments_deviations),
        # 'st_slopes': np.append(st_slopes, [np.nan] * (len(r_peaks) - len(st_slopes))),
    }

    featrues = {
        'mean_r_peaks_amplitude': np.nanmean(r_peaks_amplitudes),
        'std_r_peaks_amplitude': np.nanstd(r_peaks_amplitudes),
        'median_r_peaks_amplitude': np.nanmedian(r_peaks_amplitudes),

        'mean_rr_intervals': np.nanmean(rr_intervals),
        'std_rr_intervals': np.nanstd(rr_intervals),
        'median_rr_intervals': np.nanmedian(rr_intervals),

        'mean_heart_rate': np.nanmean(heart_rate),
        'std_heart_rate': np.nanstd(heart_rate),
        'median_heart_rate': np.nanmedian(heart_rate),

        'rmssd': rmssd,
        'nn50': nn50,
        'pnn50': pnn50,
         
        'mean_q_peaks_amplitude': np.nanmean(q_peaks_amplitudes),
        'std_q_peaks_amplitude': np.nanstd(q_peaks_amplitudes),
        'median_q_peaks_amplitude': np.nanmedian(q_peaks_amplitudes),

        'mean_s_peaks_amplitude': np.nanmean(s_peaks_amplitudes),
        'std_s_peaks_amplitude': np.nanstd(s_peaks_amplitudes),
        'median_s_peaks_amplitude': np.nanmedian(s_peaks_amplitudes),

        'mean_t_peakss_amplitude': np.nanmean(t_peaks_amplitudes),
        'std_t_peaks_amplitude': np.nanstd(t_peaks_amplitudes),
        'median_t_peaks_amplitude': np.nanmedian(t_peaks_amplitudes),

        'mean_st_segment_deviation': np.nanmean(st_segments_deviations),
        'std_st_segment_deviation': np.nanstd(st_segments_deviations),
        'median_st_segment_deviation': np.nanmedian(st_segments_deviations),

        'mean_st_slope': np.nanmean(st_slopes),
        'std_st_slope': np.nanstd(st_slopes),
        'median_st_slope': np.nanmedian(st_slopes),

        'mean_p_durations': np.nanmean(p_durations),
        'std_p_durations': np.nanstd(p_durations),
        'median_p_durations': np.nanmedian(p_durations),

        'mean_qrs_durations': np.nanmean(qrs_durations),
        'std_qrs_durations': np.nanstd(qrs_durations),
        'median_qrs_durations': np.nanmedian(qrs_durations),

        'mean_t_durations': np.nanmean(t_durations),
        'std_t_durations': np.nanstd(t_durations),
        'median_t_durations': np.nanmedian(t_durations),

        'mean_qt_intervals': np.nanmean(qt_intervals),
        'std_qt_intervals': np.nanstd(qt_intervals),
        'median_qt_intervals': np.nanmedian(qt_intervals),

        'vlf_power': vlf_power,
        'lf_power': lf_power,
        'hf_power': hf_power,
    }

    return featrues

## Main code

In [9]:
DATA_DIR = './dataset'

with open(f'{DATA_DIR}/RECORDS', 'r') as records_info_file:
    lines = records_info_file.readlines()

records = []
# Store all the record_names from the RECORD file
for record_name in lines:
    records.append(record_name.strip())

In [10]:
def get_ecg_signal(record):
    return record.p_signal


def get_signal_info(record):
    return {
        "number_of_samples": record.n_sig,
        "sampling_frequency": record.fs,
        "signal_length": record.sig_len,
        "counter_frequency": record.counter_freq,
        "leads_names": record.sig_name,
        "units": record.units,
        "adc_gain": record.adc_gain
    }


def get_patient_info(record):
    patient_info = {}
    patient_info_list = record.comments

    for info in patient_info_list:
        key, value = info.split(":", 1)
        key, value = key.strip(), value.strip()

        if value != '':
            if value == "n/a":
                patient_info[key] = 'unknown'
            else:
                patient_info[key] = value.lower()

    return patient_info


def get_xyz_file_data(file_path):
    with open(file_path, 'rb') as file:
        binary_content = file.read()

        # Define the format string for struct unpacking (3 16-bit integers)
        num_records = len(binary_content) // (3 * 2)
        format_string = f'<{num_records * 3}h'

        # Unpack the binary data
        unpacked_data = struct.unpack(format_string, binary_content)

        # Convert the unpacked data into a DataFrame
        data = {
            'x': unpacked_data[0::3],
            'y': unpacked_data[1::3],
            'z': unpacked_data[2::3]
        }

    return data

In [11]:
data = []

for record_name in records:
    record_file_path = f'{DATA_DIR}/{record_name}'
    record = wfdb.rdrecord(record_file_path)

    signal_info = get_signal_info(record)
    ecg_signal = get_ecg_signal(record)
    patient_info = get_patient_info(record)

    xyz_file_path = f'{record_file_path}.xyz'
    xyz_data = get_xyz_file_data(xyz_file_path)

    additional_diagnoses = patient_info.get('Additional diagnoses', '')
    additional_diagnoses = additional_diagnoses.split(',')
    for i in range(len(additional_diagnoses)):
        additional_diagnoses[i] = additional_diagnoses[i].strip()

    age = patient_info.get('age')
    if age == 'unknown':
        age = np.NaN
    else:
        age = int(age)

    patient_info = {
        'age': age,
        'sex': patient_info.get('sex', 'unknown'),
        'smoker': patient_info.get('Smoker', 'unknown'),
        'disease': patient_info.get('Reason for admission', 'unknown'),
        'additional_diagnoses': additional_diagnoses,
        'blood_pressure': patient_info.get('Peripheral blood Pressure (syst/diast)', 'unkown')
    }

    data.append({
        'record_name': record_name,
        'patient_info': patient_info,
        'ecg_signal': ecg_signal,
        'signals_info': signal_info,
        'xyz_data': xyz_data,
    })

In [58]:
patient_info_df = pd.DataFrame(columns=list(data[0]['patient_info'].keys()))

for record in data:
    patient_info_df.loc[record['record_name']] = record['patient_info']

# Assign a name to the index
patient_info_df.index.name = 'record_name'

In [59]:
patient_info_df

,age,sex,smoker,disease,additional_diagnoses,blood_pressure
record_name,,,,,,
patient001/s0010_re,81.0,female,no,myocardial infarction,[diabetes mellitus],140/80 mmhg
patient001/s0014lre,81.0,female,no,myocardial infarction,[diabetes mellitus],140/80 mmhg
patient001/s0016lre,81.0,female,no,myocardial infarction,[diabetes mellitus],140/80 mmhg
patient002/s0015lre,58.0,female,yes,myocardial infarction,"[arterial hypertension, adiposis hepatis, gast...",120/70 mmhg
patient003/s0017lre,63.0,male,yes,myocardial infarction,[no],unknown
...,...,...,...,...,...,...
patient292/s0555_re,66.0,male,unknown,myocardial infarction,[arterial hypertension],unknown
patient292/s0556_re,66.0,male,unknown,myocardial infarction,[arterial hypertension],unknown
patient293/s0557_re,75.0,male,unknown,myocardial infarction,[unknown],unknown


In [60]:
# Split blood pressure into two columns
patient_info_df['blood_pressure'] = patient_info_df['blood_pressure'].str.replace(' mmhg', '')

patient_info_df[['systolic_blood_pressure', 'diastolic_blood_pressure']] = patient_info_df['blood_pressure'].str.split('/', expand=True)

patient_info_df['systolic_blood_pressure'] = pd.to_numeric(
    patient_info_df['systolic_blood_pressure'], errors='coerce')
patient_info_df['diastolic_blood_pressure'] = pd.to_numeric(
    patient_info_df['diastolic_blood_pressure'], errors='coerce')

patient_info_df.drop(columns=['blood_pressure'], inplace=True)

In [61]:
patient_info_df

,age,sex,smoker,disease,additional_diagnoses,systolic_blood_pressure,diastolic_blood_pressure
record_name,,,,,,,
patient001/s0010_re,81.0,female,no,myocardial infarction,[diabetes mellitus],140.0,80.0
patient001/s0014lre,81.0,female,no,myocardial infarction,[diabetes mellitus],140.0,80.0
patient001/s0016lre,81.0,female,no,myocardial infarction,[diabetes mellitus],140.0,80.0
patient002/s0015lre,58.0,female,yes,myocardial infarction,"[arterial hypertension, adiposis hepatis, gast...",120.0,70.0
patient003/s0017lre,63.0,male,yes,myocardial infarction,[no],NaN,NaN
...,...,...,...,...,...,...,...
patient292/s0555_re,66.0,male,unknown,myocardial infarction,[arterial hypertension],NaN,NaN
patient292/s0556_re,66.0,male,unknown,myocardial infarction,[arterial hypertension],NaN,NaN
patient293/s0557_re,75.0,male,unknown,myocardial infarction,[unknown],NaN,NaN


In [88]:
diseases = ['myocardial infarction', 'healthy control',
            'valvular heart disease', 'dysrhythmia', 'heart failure (nyha 2)',
            'heart failure (nyha 3)', 'heart failure (nyha 4)',
            'palpitation', 'cardiomyopathy', 'stable angina', 'hypertrophy',
            'bundle branch block', 'unstable angina', 'myocarditis']

In [89]:
disease_encoded_df = pd.get_dummies(patient_info_df['disease'])
disease_encoded_df.drop(columns=['unknown'], inplace=True)

In [90]:
disease_encoded_df

,bundle branch block,cardiomyopathy,dysrhythmia,healthy control,heart failure (nyha 2),heart failure (nyha 3),heart failure (nyha 4),hypertrophy,myocardial infarction,myocarditis,palpitation,stable angina,unstable angina,valvular heart disease
record_name,,,,,,,,,,,,,,
patient001/s0010_re,False,False,False,False,False,False,False,False,True,False,False,False,False,False
patient001/s0014lre,False,False,False,False,False,False,False,False,True,False,False,False,False,False
patient001/s0016lre,False,False,False,False,False,False,False,False,True,False,False,False,False,False
patient002/s0015lre,False,False,False,False,False,False,False,False,True,False,False,False,False,False
patient003/s0017lre,False,False,False,False,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
patient292/s0555_re,False,False,False,False,False,False,False,False,True,False,False,False,False,False
patient292/s0556_re,False,False,False,False,False,False,False,False,True,False,False,False,False,False
patient293/s0557_re,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [91]:
disease_encoded_df = disease_encoded_df.astype(int)

In [92]:
disease_encoded_df

,bundle branch block,cardiomyopathy,dysrhythmia,healthy control,heart failure (nyha 2),heart failure (nyha 3),heart failure (nyha 4),hypertrophy,myocardial infarction,myocarditis,palpitation,stable angina,unstable angina,valvular heart disease
record_name,,,,,,,,,,,,,,
patient001/s0010_re,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient001/s0014lre,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient001/s0016lre,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient002/s0015lre,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient003/s0017lre,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
patient292/s0555_re,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient292/s0556_re,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient293/s0557_re,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [93]:
# Iterate over each row
for index, row in patient_info_df.iterrows():
    additional_diagnoses = row['additional_diagnoses']
    matched_diseases = [
        diagnosis for diagnosis in additional_diagnoses if diagnosis in diseases
    ]
    if matched_diseases:
        print(f"Row {index}: Additional diagnoses matched with {', '.join(matched_diseases)}")
        
        for disease in matched_diseases:
            disease_encoded_df.loc[index, disease] = 1

Row patient112/s0169_re: Additional diagnoses matched with palpitation
Row patient114/s0012_re: Additional diagnoses matched with stable angina
Row patient127/s0342lre: Additional diagnoses matched with heart failure (nyha 3)
Row patient127/s0383lre: Additional diagnoses matched with heart failure (nyha 3)
Row patient133/s0393lre: Additional diagnoses matched with palpitation
Row patient142/s0351lre: Additional diagnoses matched with palpitation
Row patient147/s0211_re: Additional diagnoses matched with palpitation
Row patient205/s0426_re: Additional diagnoses matched with bundle branch block
Row patient207/s0428_re: Additional diagnoses matched with bundle branch block
Row patient215/s0437_re: Additional diagnoses matched with bundle branch block
Row patient232/s0456_re: Additional diagnoses matched with bundle branch block
Row patient269/s0508_re: Additional diagnoses matched with bundle branch block


In [94]:
# Calculate the sum of diseases for each patient
disease_sum = disease_encoded_df.sum(axis=1)

# Filter the DataFrame to get records with more than one disease
multiple_diseases_df = disease_encoded_df[disease_sum > 1]

In [95]:
multiple_diseases_df

,bundle branch block,cardiomyopathy,dysrhythmia,healthy control,heart failure (nyha 2),heart failure (nyha 3),heart failure (nyha 4),hypertrophy,myocardial infarction,myocarditis,palpitation,stable angina,unstable angina,valvular heart disease
record_name,,,,,,,,,,,,,,
patient112/s0169_re,0,0,1,0,0,0,0,0,0,0,1,0,0,0
patient114/s0012_re,0,0,0,0,0,0,0,0,0,0,0,1,0,1
patient127/s0342lre,0,1,0,0,0,1,0,0,0,0,0,0,0,0
patient127/s0383lre,0,1,0,0,0,1,0,0,0,0,0,0,0,0
patient133/s0393lre,0,0,1,0,0,0,0,0,0,0,1,0,0,0
patient142/s0351lre,0,0,0,0,0,0,0,0,1,0,1,0,0,0
patient147/s0211_re,0,0,1,0,0,0,0,0,0,0,1,0,0,0
patient205/s0426_re,1,0,0,0,0,0,0,0,1,0,0,0,0,0
patient207/s0428_re,1,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
# patient_info_df.to_csv('patient_info.csv')

In [96]:
numerical_p_info_df = patient_info_df.copy()

In [97]:
# Map 'sex' to binary values: 'male' -> 0, 'female' -> 1
numerical_p_info_df['sex'] = patient_info_df['sex'].map(
    {'male': 0, 'female': 1, 'unknown': None}
)
numerical_p_info_df['smoker'] = patient_info_df['smoker'].map(
    {'no': 0, 'yes': 1, 'unknown': None}
)

In [98]:
numerical_p_info_df

,age,sex,smoker,disease,additional_diagnoses,systolic_blood_pressure,diastolic_blood_pressure
record_name,,,,,,,
patient001/s0010_re,81.0,1.0,0.0,myocardial infarction,[diabetes mellitus],140.0,80.0
patient001/s0014lre,81.0,1.0,0.0,myocardial infarction,[diabetes mellitus],140.0,80.0
patient001/s0016lre,81.0,1.0,0.0,myocardial infarction,[diabetes mellitus],140.0,80.0
patient002/s0015lre,58.0,1.0,1.0,myocardial infarction,"[arterial hypertension, adiposis hepatis, gast...",120.0,70.0
patient003/s0017lre,63.0,0.0,1.0,myocardial infarction,[no],NaN,NaN
...,...,...,...,...,...,...,...
patient292/s0555_re,66.0,0.0,NaN,myocardial infarction,[arterial hypertension],NaN,NaN
patient292/s0556_re,66.0,0.0,NaN,myocardial infarction,[arterial hypertension],NaN,NaN
patient293/s0557_re,75.0,0.0,NaN,myocardial infarction,[unknown],NaN,NaN


In [99]:
numerical_p_info_df.drop(columns=['disease', 'additional_diagnoses'], inplace=True)

In [100]:
numerical_p_info_df

,age,sex,smoker,systolic_blood_pressure,diastolic_blood_pressure
record_name,,,,,
patient001/s0010_re,81.0,1.0,0.0,140.0,80.0
patient001/s0014lre,81.0,1.0,0.0,140.0,80.0
patient001/s0016lre,81.0,1.0,0.0,140.0,80.0
patient002/s0015lre,58.0,1.0,1.0,120.0,70.0
patient003/s0017lre,63.0,0.0,1.0,NaN,NaN
...,...,...,...,...,...
patient292/s0555_re,66.0,0.0,NaN,NaN,NaN
patient292/s0556_re,66.0,0.0,NaN,NaN,NaN
patient293/s0557_re,75.0,0.0,NaN,NaN,NaN


In [101]:
disease_with_patient_info_df = pd.concat([numerical_p_info_df, disease_encoded_df], axis=1)

In [102]:
disease_with_patient_info_df

,age,sex,smoker,systolic_blood_pressure,diastolic_blood_pressure,bundle branch block,cardiomyopathy,dysrhythmia,healthy control,heart failure (nyha 2),heart failure (nyha 3),heart failure (nyha 4),hypertrophy,myocardial infarction,myocarditis,palpitation,stable angina,unstable angina,valvular heart disease
record_name,,,,,,,,,,,,,,,,,,,
patient001/s0010_re,81.0,1.0,0.0,140.0,80.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient001/s0014lre,81.0,1.0,0.0,140.0,80.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient001/s0016lre,81.0,1.0,0.0,140.0,80.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient002/s0015lre,58.0,1.0,1.0,120.0,70.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient003/s0017lre,63.0,0.0,1.0,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
patient292/s0555_re,66.0,0.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient292/s0556_re,66.0,0.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0
patient293/s0557_re,75.0,0.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
# from collections import Counter

# # Extract all additional diagnoses from the 'additional_diagnoses' column
# additional_diagnoses_list = patient_info_df['additional_diagnoses']

# # Remove duplicates and empty strings
# additional_diagnoses_list = additional_diagnoses_list[additional_diagnoses_list != ''].drop_duplicates()

# # Convert the list of diseases to a set for faster membership testing
# diseases_set = set(diseases)

# # Filter out additional diagnoses not in the list of diseases you mentioned
# unrecognized_diagnoses = additional_diagnoses_list[~additional_diagnoses_list.isin(
#     diseases_set)]

# # Flatten the list of lists into a single list
# flat_unrecognized_diagnoses = [
#     item for sublist in unrecognized_diagnoses for item in sublist]

# # Count occurrences of each unrecognized diagnosis
# unrecognized_diagnoses_counts = Counter(flat_unrecognized_diagnoses)

# # Print unrecognized diagnoses and their counts
# for diagnosis, count in unrecognized_diagnoses_counts.items():
#     print(f"Diagnosis: {diagnosis}, Count: {count}")

In [15]:
all_leads_features = []

for i in range(len(data)):
    record = data[i]
    all_leads_signals = record['ecg_signal']
    patient_info = record['patient_info']

    leads = record['signals_info']['leads_names']

    all_leads_df = pd.DataFrame(all_leads_signals, columns=leads)
    all_leads_df['time'] = [i for i in range(len(all_leads_df))]

    features_dict = {}

    for lead in leads:
        # Here we are only taking the lead 1 and extracting all the data from it
        ecg_signal = all_leads_df[lead].to_numpy()
        fs = record['signals_info']['sampling_frequency']

        filtered_signal = filter_signal(ecg_signal, fs)

        df = pd.DataFrame(filtered_signal, columns=[lead])
        df['time'] = [(i / fs) * 1000 for i in range(len(filtered_signal))]

        # print(df.head())

        plot_label = f"{i+1}: {record['record_name']} - Lead {lead} Signal, Age - {patient_info['age']}, Disease - {patient_info['disease']}"
        # plot_interactive_ecg(df, leads=['i'], label=plot_label, start=0, end=min(5000, df.shape[0]))

        print(plot_label)
        features = extract_features(filtered_signal, fs)
        features_dict[lead] = features
        # print(features)
        # print()

    all_leads_features.append(features_dict)

1: patient001/s0010_re - Lead i Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead ii Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead iii Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead avr Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead avl Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead avf Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead v1 Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead v2 Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead v3 Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead v4 Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead v5 Signal, Age - 81, Disease - myocardial infarction
1: patient001/s0010_re - Lead v6 Signal, Age - 81, Disease - myocardial infarction
1

C:\Users\Akash\AppData\Local\Temp\ipykernel_6744\1066869016.py:58: RuntimeWarning: Mean of empty slice
  st_segment_mean = np.nanmean(
C:\Users\Akash\AppData\Local\Temp\ipykernel_6744\816397341.py:17: RuntimeWarning: invalid value encountered in scalar divide
  slope = (ecg_signal[st_segment_end] - ecg_signal[st_segment_start]
C:\Users\Akash\AppData\Local\Temp\ipykernel_6744\1066869016.py:58: RuntimeWarning: Mean of empty slice
  st_segment_mean = np.nanmean(


2: patient001/s0014lre - Lead v1 Signal, Age - 81, Disease - myocardial infarction
2: patient001/s0014lre - Lead v2 Signal, Age - 81, Disease - myocardial infarction
2: patient001/s0014lre - Lead v3 Signal, Age - 81, Disease - myocardial infarction
2: patient001/s0014lre - Lead v4 Signal, Age - 81, Disease - myocardial infarction
2: patient001/s0014lre - Lead v5 Signal, Age - 81, Disease - myocardial infarction
2: patient001/s0014lre - Lead v6 Signal, Age - 81, Disease - myocardial infarction
2: patient001/s0014lre - Lead vx Signal, Age - 81, Disease - myocardial infarction
2: patient001/s0014lre - Lead vy Signal, Age - 81, Disease - myocardial infarction
2: patient001/s0014lre - Lead vz Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead i Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead ii Signal, Age - 81, Disease - myocardial infarction


C:\Users\Akash\AppData\Local\Temp\ipykernel_6744\1066869016.py:58: RuntimeWarning: Mean of empty slice
  st_segment_mean = np.nanmean(


3: patient001/s0016lre - Lead iii Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead avr Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead avl Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead avf Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead v1 Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead v2 Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead v3 Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead v4 Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead v5 Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead v6 Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead vx Signal, Age - 81, Disease - myocardial infarction
3: patient001/s0016lre - Lead vy Signal, Age - 81, Disease - myocardial infarction


...

C:\Users\Akash\AppData\Local\Temp\ipykernel_6744\1066869016.py:58: RuntimeWarning: Mean of empty slice
  st_segment_mean = np.nanmean(


548: patient293/s0558_re - Lead v5 Signal, Age - 75, Disease - myocardial infarction
548: patient293/s0558_re - Lead v6 Signal, Age - 75, Disease - myocardial infarction
548: patient293/s0558_re - Lead vx Signal, Age - 75, Disease - myocardial infarction
548: patient293/s0558_re - Lead vy Signal, Age - 75, Disease - myocardial infarction
548: patient293/s0558_re - Lead vz Signal, Age - 75, Disease - myocardial infarction
549: patient294/s0559_re - Lead i Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead ii Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead iii Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead avr Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead avl Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead avf Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead v1 Signal, Age - 61, Disease -

C:\Users\Akash\AppData\Local\Temp\ipykernel_6744\1066869016.py:58: RuntimeWarning: Mean of empty slice
  st_segment_mean = np.nanmean(
C:\Users\Akash\AppData\Local\Temp\ipykernel_6744\816397341.py:17: RuntimeWarning: invalid value encountered in scalar divide
  slope = (ecg_signal[st_segment_end] - ecg_signal[st_segment_start]
C:\Users\Akash\AppData\Local\Temp\ipykernel_6744\1066869016.py:58: RuntimeWarning: Mean of empty slice
  st_segment_mean = np.nanmean(
C:\Users\Akash\AppData\Local\Temp\ipykernel_6744\1066869016.py:58: RuntimeWarning: Mean of empty slice
  st_segment_mean = np.nanmean(


549: patient294/s0559_re - Lead v2 Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead v3 Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead v4 Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead v5 Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead v6 Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead vx Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead vy Signal, Age - 61, Disease - myocardial infarction
549: patient294/s0559_re - Lead vz Signal, Age - 61, Disease - myocardial infarction


C:\Users\Akash\AppData\Local\Temp\ipykernel_6744\1066869016.py:58: RuntimeWarning: Mean of empty slice
  st_segment_mean = np.nanmean(


In [103]:
print(all_leads_features[0])

{'i': {'mean_r_peaks_amplitude': 0.44325412885766763, 'std_r_peaks_amplitude': 0.013022761426208632, 'median_r_peaks_amplitude': 0.44374749887912607, 'mean_rr_intervals': 733.7647058823529, 'std_rr_intervals': 9.43624267883605, 'median_rr_intervals': 733.0, 'mean_heart_rate': 81.78360697085667, 'std_heart_rate': 1.051872320402508, 'median_heart_rate': 81.85538881309687, 'rmssd': 11.131936040060596, 'nn50': 0, 'pnn50': 0.0, 'mean_q_peaks_amplitude': -0.16749313601043597, 'std_q_peaks_amplitude': 0.015854414727295693, 'median_q_peaks_amplitude': -0.16703138166113318, 'mean_s_peaks_amplitude': -0.44324896352157095, 'std_s_peaks_amplitude': 0.014335479030431397, 'median_s_peaks_amplitude': -0.443634720241352, 'mean_t_peakss_amplitude': 0.13838066577738936, 'std_t_peaks_amplitude': 0.013036533290051102, 'median_t_peaks_amplitude': 0.13838152991552574, 'mean_st_segment_deviation': -0.5213248559557189, 'std_st_segment_deviation': 0.01674098746777758, 'median_st_segment_deviation': -0.52001636

In [105]:
combined_data = []

for i in range(len(all_leads_features)):
    row = all_leads_features[i]
    info = disease_with_patient_info_df.iloc[i].to_dict()
    for lead, features in row.items():
        combined_data.append({**info, **features})

In [106]:
len(combined_data)

8235

In [107]:
master_df = pd.DataFrame(combined_data)

In [108]:
master_df

,age,sex,smoker,systolic_blood_pressure,diastolic_blood_pressure,bundle branch block,cardiomyopathy,dysrhythmia,healthy control,heart failure (nyha 2),...,median_qrs_durations,mean_t_durations,std_t_durations,median_t_durations,mean_qt_intervals,std_qt_intervals,median_qt_intervals,vlf_power,lf_power,hf_power
0,81.0,1.0,0.0,140.0,80.0,0.0,0.0,0.0,0.0,0.0,...,87.0,277.865385,6.348921,279.0,306.423077,6.298187,307.0,0.0,0.0,0.0
1,81.0,1.0,0.0,140.0,80.0,0.0,0.0,0.0,0.0,0.0,...,38.0,103.884615,14.653240,100.5,121.750000,18.770758,123.0,0.0,0.0,0.0
2,81.0,1.0,0.0,140.0,80.0,0.0,0.0,0.0,0.0,0.0,...,14.0,27.269231,0.879584,27.0,41.288462,1.214283,42.0,0.0,0.0,0.0
3,81.0,1.0,0.0,140.0,80.0,0.0,0.0,0.0,0.0,0.0,...,1.0,57.596154,1.348213,57.5,58.615385,1.346703,59.0,0.0,0.0,0.0
4,81.0,1.0,0.0,140.0,80.0,0.0,0.0,0.0,0.0,0.0,...,90.0,243.923077,18.162563,237.0,273.538462,36.812655,297.5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8230,61.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,100.0,249.807339,8.996918,254.0,281.513761,9.046886,285.0,0.0,0.0,0.0
8231,61.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,101.0,267.342282,3.644932,268.0,299.201342,3.656345,299.0,0.0,0.0,0.0
8232,61.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,99.0,263.449664,2.361424,263.0,295.919463,2.341676,296.0,0.0,0.0,0.0
8233,61.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,94.0,155.228188,89.717313,116.0,183.577181,89.811524,144.0,0.0,0.0,0.0


In [109]:
master_df.to_csv("master_data_all_leads.csv")

In [110]:
df = master_df.copy()

In [111]:
# Features
X = df.drop(columns=disease_encoded_df.columns)

# Targets
y = df[disease_encoded_df.columns]

In [112]:
X

,age,sex,smoker,systolic_blood_pressure,diastolic_blood_pressure,mean_r_peaks_amplitude,std_r_peaks_amplitude,median_r_peaks_amplitude,mean_rr_intervals,std_rr_intervals,...,median_qrs_durations,mean_t_durations,std_t_durations,median_t_durations,mean_qt_intervals,std_qt_intervals,median_qt_intervals,vlf_power,lf_power,hf_power
0,81.0,1.0,0.0,140.0,80.0,0.443254,0.013023,0.443747,733.764706,9.436243,...,87.0,277.865385,6.348921,279.0,306.423077,6.298187,307.0,0.0,0.0,0.0
1,81.0,1.0,0.0,140.0,80.0,-0.218650,0.026060,-0.220316,733.784314,9.421115,...,38.0,103.884615,14.653240,100.5,121.750000,18.770758,123.0,0.0,0.0,0.0
2,81.0,1.0,0.0,140.0,80.0,-0.239967,0.020062,-0.237549,733.764706,9.398766,...,14.0,27.269231,0.879584,27.0,41.288462,1.214283,42.0,0.0,0.0,0.0
3,81.0,1.0,0.0,140.0,80.0,-0.197383,0.013107,-0.197509,733.764706,9.434165,...,1.0,57.596154,1.348213,57.5,58.615385,1.346703,59.0,0.0,0.0,0.0
4,81.0,1.0,0.0,140.0,80.0,-0.052284,0.209993,-0.139005,733.725490,31.606700,...,90.0,243.923077,18.162563,237.0,273.538462,36.812655,297.5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8230,61.0,1.0,NaN,NaN,NaN,1.186274,0.051521,1.183949,1088.185185,505.843797,...,100.0,249.807339,8.996918,254.0,281.513761,9.046886,285.0,0.0,0.0,0.0
8231,61.0,1.0,NaN,NaN,NaN,0.867600,0.018980,0.866156,804.932432,59.311097,...,101.0,267.342282,3.644932,268.0,299.201342,3.656345,299.0,0.0,0.0,0.0
8232,61.0,1.0,NaN,NaN,NaN,1.039915,0.021605,1.038964,804.932432,59.458441,...,99.0,263.449664,2.361424,263.0,295.919463,2.341676,296.0,0.0,0.0,0.0
8233,61.0,1.0,NaN,NaN,NaN,0.597420,0.022094,0.596290,804.925676,58.814998,...,94.0,155.228188,89.717313,116.0,183.577181,89.811524,144.0,0.0,0.0,0.0


In [113]:
y

,bundle branch block,cardiomyopathy,dysrhythmia,healthy control,heart failure (nyha 2),heart failure (nyha 3),heart failure (nyha 4),hypertrophy,myocardial infarction,myocarditis,palpitation,stable angina,unstable angina,valvular heart disease
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [125]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [115]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [116]:
# Initialize the scaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Create a pipeline that first imputes missing values and then scales the data
pipeline = Pipeline([
    # You can choose 'median' or 'most_frequent' as well
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

X_train_scaled = pipeline.fit_transform(X_train)
X_test_scaled = pipeline.transform(X_test)

In [117]:
y_train

,bundle branch block,cardiomyopathy,dysrhythmia,healthy control,heart failure (nyha 2),heart failure (nyha 3),heart failure (nyha 4),hypertrophy,myocardial infarction,myocarditis,palpitation,stable angina,unstable angina,valvular heart disease
2522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5896,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7704,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4901,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5191,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5390,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [118]:
from keras import Sequential
from keras.layers import Dense, Input

# Define the neural network model
model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(14, activation='sigmoid')  # The number of neurons here must match the number of diseases which is 14 in our case
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2
)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)

print('Test Accuracy:', test_acc)

Epoch 1/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5663 - loss: 0.3870 - val_accuracy: 0.6851 - val_loss: 0.1173
Epoch 2/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7198 - loss: 0.1089 - val_accuracy: 0.7618 - val_loss: 0.0978
Epoch 3/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.7645 - loss: 0.0897 - val_accuracy: 0.7769 - val_loss: 0.0874
Epoch 4/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7843 - loss: 0.0798 - val_accuracy: 0.7997 - val_loss: 0.0787
Epoch 5/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - accuracy: 0.7984 - loss: 0.0715 - val_accuracy: 0.8156 - val_loss: 0.0725
Epoch 6/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - accuracy: 0.8100 - loss: 0.0653 - val_accuracy: 0.8202 - val_loss: 0.0693
Epoch 7/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8196 - loss: 0.0602 - val_accuracy: 0.8278 - val_loss: 0.0666
Epoch 8/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.8269 - loss: 0.0577 - val_accu

In [119]:
# Make predictions on the validation set
y_pred = model.predict(X_test)

52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [120]:
from sklearn.metrics import classification_report

# Predict the probabilities on the test set
y_pred_prob = model.predict(X_test_scaled)

# Binarize the predictions with a threshold of 0.5
y_pred = (y_pred_prob > 0.5).astype(int)

# Print the classification report
print(classification_report(y_test, y_pred, target_names=[i for i in disease_encoded_df.columns], zero_division=0))

52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step
                        precision    recall  f1-score   support

   bundle branch block       0.66      0.44      0.52        62
        cardiomyopathy       0.89      0.37      0.52        46
           dysrhythmia       0.83      0.70      0.76        57
       healthy control       0.87      0.91      0.89       227
heart failure (nyha 2)       1.00      1.00      1.00         3
heart failure (nyha 3)       1.00      0.83      0.91        12
heart failure (nyha 4)       0.00      0.00      0.00         4
           hypertrophy       0.59      0.65      0.62        20
 myocardial infarction       0.96      0.94      0.95      1103
           myocarditis       0.31      0.50      0.38         8
           palpitation       0.64      0.54      0.58        13
         stable angina       1.00      0.14      0.25        14
       unstable angina       1.00      1.00      1.00         2
valvular heart disease       0.88      0.65      0.75        2

In [126]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Initialize the base model
base_model = RandomForestClassifier(random_state=42)

# Initialize the multi-output classifier
model = MultiOutputClassifier(base_model, n_jobs=-1)

In [123]:
# Train the model
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

In [124]:
# Evaluate the model for each disease
from sklearn.metrics import accuracy_score, classification_report

for i, disease in enumerate(y.columns):
    print(f"Evaluating model for disease: {disease}")
    print(f"Accuracy: {accuracy_score(y_test.iloc[:, i], y_pred[:, i])}")
    print("Classification Report:")
    print(classification_report(
        y_test.iloc[:, i], y_pred[:, i], zero_division=0)
    )
    print("\n")

Evaluating model for disease: bundle branch block
Accuracy: 0.9848208864602307
Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      1585
         1.0       0.97      0.61      0.75        62

    accuracy                           0.98      1647
   macro avg       0.98      0.81      0.87      1647
weighted avg       0.98      0.98      0.98      1647



Evaluating model for disease: cardiomyopathy
Accuracy: 0.9878567091681846
Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      1601
         1.0       1.00      0.57      0.72        46

    accuracy                           0.99      1647
   macro avg       0.99      0.78      0.86      1647
weighted avg       0.99      0.99      0.99      1647



Evaluating model for disease: dysrhythmia
Accuracy: 0.9914996964177292
Classification Report:
              precision    recall  f1-score   sup

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Compute class weights for each disease
class_weights_dict = {}
for disease in y_train.columns:
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.array([0, 1]),
        y=y_train[disease].values
    )
    class_weights_dict[disease] = {0: class_weights[0], 1: class_weights[1]}

# Initialize the base model
base_model = RandomForestClassifier(random_state=42)

# Custom multi-output classifier with class weights
class WeightedMultiOutputClassifier(MultiOutputClassifier):
    def fit(self, X, y, sample_weight=None):
        result = super().fit(X, y, sample_weight=sample_weight)
        # Set class weights for each estimator
        for i, estimator in enumerate(self.estimators_):
            estimator.class_weight = class_weights_dict[y.columns[i]]
        return result


# Initialize the custom multi-output classifier
model = WeightedMultiOutputClassifier(base_model, n_jobs=-1)

# Train the model
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
for i, disease in enumerate(y_test.columns):
    print(f"Evaluating model for disease: {disease}")
    print(f"Accuracy: {accuracy_score(y_test.iloc[:, i], y_pred[:, i])}")
    print("Classification Report:")
    print(classification_report(
        y_test.iloc[:, i], y_pred[:, i], zero_division=0))
    print("\n")

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# List of classifiers to test
classifiers = {
    "SVM": SVC(probability=True, random_state=42),
    "k-NN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
}

# Compute class weights for each disease
class_weights_dict = {}
for disease in y_train.columns:
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.array([0, 1]),
        y=y_train[disease].values
    )
    class_weights_dict[disease] = {0: class_weights[0], 1: class_weights[1]}


results = []

# Function to fit and evaluate a classifier
def evaluate_classifier(name, classifier):
    print(f"Evaluating classifier: {name}")

    # Custom multi-output classifier with class weights
    class WeightedMultiOutputClassifier(MultiOutputClassifier):
        def fit(self, X, y, sample_weight=None):
            result = super().fit(X, y, sample_weight=sample_weight)
            # Set class weights for each estimator
            for i, estimator in enumerate(self.estimators_):
                estimator.class_weight = class_weights_dict[y.columns[i]]
            return result

    model = WeightedMultiOutputClassifier(classifier, n_jobs=-1)
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)

    # Evaluate the model
    for i, disease in enumerate(y_test.columns):
        print(f"Evaluating model for disease: {disease}")
        print(f"Accuracy: {accuracy_score(y_test.iloc[:, i], y_pred[:, i])}")
        print("Classification Report:")
        print(classification_report(
            y_test.iloc[:, i], y_pred[:, i], zero_division=0))
        print("\n")

        accuracy = accuracy_score(y_test.iloc[:, i], y_pred[:, i])
        precision = precision_score(
            y_test.iloc[:, i], y_pred[:, i], zero_division=0)
        recall = recall_score(y_test.iloc[:, i], y_pred[:, i], zero_division=0)
        f1 = f1_score(y_test.iloc[:, i], y_pred[:, i], zero_division=0)

        # Store results in the DataFrame
        results.append({
            'Model': name,
            'Disease': disease,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1
        })


# Loop through the classifiers and evaluate each one
for name, classifier in classifiers.items():
    evaluate_classifier(name, classifier)

# Convert results to DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df